# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [4]:
def func_query_janela(lista_janela, feat_num_lista, id, safra_ref, target, tb_feat, safra):
    query_janela = ""

    for n in lista_janela:
        query_variaveis_numericas=""
        for i in feat_num_lista:
            query_variaveis_numericas +=f"""
             -- Criação de variáveis numéricas a partir da coluna {i} para a janela {n}
            sum(tb_feat.{i}) AS {i}_SUM_{n}M,
            min(tb_feat.{i}) AS {i}_MIN_{n}M,
            max(tb_feat.{i}) AS {i}_MAX_{n}M,
            avg(tb_feat.{i}) AS {i}_AGV_{n}M,
        """
        query_variaveis_numericas = query_variaveis_numericas.rstrip(', \n')

        query_janela += f"""
        -- Criação de variáveis de janela de {n}M
        tb_janela_{n}M as(
            SELECT 
                tb_public.{id},
                tb_public.{safra_ref},
                tb_public.{target},
                {query_variaveis_numericas}

            FROM tb_public
            LEFT JOIN {tb_feat} 
            ON  tb_public.{id} = tb_feat.{id}
            AND (strftime('%Y-%m-%d', date({tb_feat}.{safra}, '+{n} months')) >= tb_public.{safra_ref})
                AND ({tb_feat}.{safra} < tb_public.{safra_ref})
            GROUP BY tb_public.{id}, tb_public.{safra_ref}
        ),
        """
    return query_janela

In [5]:
def func_query_join(lista_janela, id, safra_ref):
    query_join= ""
    for i in lista_janela:
        query_join += f"""
        LEFT JOIN tb_janela_{i}M
            ON tb_public.{id} = tb_janela_{i}M.{id}
            AND tb_public.{safra_ref} = tb_janela_{i}M.{safra_ref}
    """
    return query_join

In [8]:
def func_query_final(tb_publico, lista_janela,feat_num_lista, id, safra_ref, target, tb_feat, safra):
    query_final = f"""
    with 
    tb_public as (
        SELECT 
            *
        FROM {tb_publico}
    ),
    {func_query_janela(lista_janela, feat_num_lista, id, safra_ref, target, tb_feat, safra)}

    tb_join as (
        SELECT 
            *
        FROM tb_public 
        {func_query_join(lista_janela, id, safra_ref)}
    )
        
    SELECT 
        *
    FROM tb_join
    """
    return query_final

In [10]:
tb_publico='tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
target = 'Target'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,3,6]
query = func_query_final(tb_publico, lista_janela,feat_num_lista, id, safra_ref, target, tb_feat, safra)

In [11]:
print(query)


    with 
    tb_public as (
        SELECT 
            *
        FROM tb_spine
    ),
    
        -- Criação de variáveis de janela de 1M
        tb_janela_1M as(
            SELECT 
                tb_public.ID,
                tb_public.SAFRA_REF,
                tb_public.Target,
                
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            sum(tb_feat.FEAT_NUM1) AS FEAT_NUM1_SUM_1M,
            min(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MIN_1M,
            max(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MAX_1M,
            avg(tb_feat.FEAT_NUM1) AS FEAT_NUM1_AGV_1M,
        
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            sum(tb_feat.FEAT_NUM2) AS FEAT_NUM2_SUM_1M,
            min(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MIN_1M,
            max(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MAX_1M,
            avg(tb_feat.FEAT_NUM2) AS FEAT_NUM2_AGV_1M

            FROM tb_public
            LEFT JOIN tb_fe

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()